In [11]:
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf

df = pd.read_csv('dataset_1654342.csv')
df.head()

,transaction_id,transaction_date,transaction_time,transaction_qty,store_id,store_location,unit_price,product_category,product_type
0,1,06/08/2023,17:41:54,1.0,2,Groningen,3.25,Tea,Chai tea
1,2,06/10/2023,11:30:46,2.0,2,Groningen,4.50,Chocolate,Hot chocolate
2,3,06/06/2023,09:19:26,3.0,2,Groningen,3.00,Tea,Black tea
3,4,06/10/2023,17:25:49,1.0,3,Breda,4.50,Chocolate,Hot chocolate
4,5,06/05/2023,14:55:12,1.0,1,Amsterdam,4.50,Chocolate,Hot chocolate


In [12]:
product_category_dummies = pd.get_dummies(df['product_category'])
df_product_category_dummies = pd.concat([df,product_category_dummies],axis=1)
df_product_category_dummies.head()

,transaction_id,transaction_date,transaction_time,transaction_qty,store_id,store_location,unit_price,product_category,product_type,Bakery,Chocolate,Coffee,Tea
0,1,06/08/2023,17:41:54,1.0,2,Groningen,3.25,Tea,Chai tea,False,False,False,True
1,2,06/10/2023,11:30:46,2.0,2,Groningen,4.50,Chocolate,Hot chocolate,False,True,False,False
2,3,06/06/2023,09:19:26,3.0,2,Groningen,3.00,Tea,Black tea,False,False,False,True
3,4,06/10/2023,17:25:49,1.0,3,Breda,4.50,Chocolate,Hot chocolate,False,True,False,False
4,5,06/05/2023,14:55:12,1.0,1,Amsterdam,4.50,Chocolate,Hot chocolate,False,True,False,False


In [13]:
df_product_category_dummies['revenue_per_transaction'] = df_product_category_dummies['transaction_qty'] * df_product_category_dummies['unit_price']

In [14]:
df_product_category_dummies.head()

,transaction_id,transaction_date,transaction_time,transaction_qty,store_id,store_location,unit_price,product_category,product_type,Bakery,Chocolate,Coffee,Tea,revenue_per_transaction
0,1,06/08/2023,17:41:54,1.0,2,Groningen,3.25,Tea,Chai tea,False,False,False,True,3.25
1,2,06/10/2023,11:30:46,2.0,2,Groningen,4.50,Chocolate,Hot chocolate,False,True,False,False,9.00
2,3,06/06/2023,09:19:26,3.0,2,Groningen,3.00,Tea,Black tea,False,False,False,True,9.00
3,4,06/10/2023,17:25:49,1.0,3,Breda,4.50,Chocolate,Hot chocolate,False,True,False,False,4.50
4,5,06/05/2023,14:55:12,1.0,1,Amsterdam,4.50,Chocolate,Hot chocolate,False,True,False,False,4.50


In [15]:
filtered_df_product_category_dummies = df_product_category_dummies.dropna(subset=['transaction_qty','revenue_per_transaction'])

In [19]:
model1 = smf.ols('revenue_per_transaction ~ Bakery + Chocolate + Coffee + Tea',data = filtered_df_product_category_dummies).fit()
print(model1.summary())

                               OLS Regression Results                              
Dep. Variable:     revenue_per_transaction   R-squared:                       0.044
Model:                                 OLS   Adj. R-squared:                  0.044
Method:                      Least Squares   F-statistic:                     114.7
Date:                     Mon, 25 Mar 2024   Prob (F-statistic):           1.34e-72
Time:                             10:22:55   Log-Likelihood:                -22367.
No. Observations:                     7429   AIC:                         4.474e+04
Df Residuals:                         7425   BIC:                         4.477e+04
Df Model:                                3                                         
Covariance Type:                 nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------

In [25]:
# Looking at the regression model it means that especially Coffee and Chocolate have the most positive effects on the revenue per transaction. And those results are statistically significant. 
# Underneath we can also see that these categories have the highest gross revenue.
# One thing to note is that the R-squared is only 0.044 so this means these results only have an affect on the revenue_per_transaction of 4.4% which is not a lot.


In [26]:
# Sum the revenue per product category
total_revenue_per_category = df_product_category_dummies.groupby('product_category')['revenue_per_transaction'].sum()

total_revenue_per_category

product_category
Bakery       12307.00
Chocolate    16978.50
Coffee       14889.85
Tea          11942.60
Name: revenue_per_transaction, dtype: float64